In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [42]:
import pandas as pd

marvel_comics = pd.read_csv("marvel_comics_cleaned.csv")

In [43]:
marvel_comics

,comic_name,active_years,issue_title,publish_date,issue_description,penciler,writer,cover_artist,Imprint,Format,Rating,Price,id,tagged_description
0,A Year of Marvels: April Infinite Comic (2016),(2016),A Year of Marvels: April Infinite Comic (2016) #1,"April 01, 2016",The Infinite Comic that will have everyone tal...,Yves Bigerel,Yves Bigerel,Jamal Campbell,Marvel Universe,Infinite Comic,Rated T+,Free,0,0 The Infinite Comic that will have everyone t...
1,A Year of Marvels: August Infinite Comic (2016),(2016),A Year of Marvels: August Infinite Comic (2016...,"August 10, 2016","It’s August, and Nick Fury is just in time to ...",Jamal Campbell,"Chris Sims, Chad Bowers",NaN,Marvel Universe,Infinite Comic,NaN,Free,1,"1 It’s August, and Nick Fury is just in time t..."
2,A Year of Marvels: February Infinite Comic (2016),(2016),A Year of Marvels: February Infinite Comic (20...,"February 10, 2016",Join us in a brand new Marvel comics adventure...,"Danilo S. Beyruth, M Mast",Ryan North,NaN,Marvel Universe,Infinite Comic,Rated T+,Free,2,2 Join us in a brand new Marvel comics adventu...
3,A Year of Marvels: July Infinite Comic (2016),(2016),A Year of Marvels: July Infinite Comic (2016) #1,"June 29, 2016",Celebrating the Fourth of July is complicated ...,Juanan Ramirez,Chuck Wendig,Jamal Campbell,Marvel Universe,Infinite Comic,NaN,Free,3,3 Celebrating the Fourth of July is complicate...
4,A Year of Marvels: June Infinite Comic (2016),(2016),A Year of Marvels: June Infinite Comic (2016) #1,"June 15, 2016",Sam Alexander’s finding it hard to cope with t...,Diego Olortegui,Paul Allor,Jamal Campbell,Marvel Universe,Infinite Comic,NaN,Free,4,4 Sam Alexander’s finding it hard to cope with...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23472,Ziggy Pig - Silly Seal Comics (2019),(2019),Ziggy Pig - Silly Seal Comics (2019) #1,"March 06, 2019",NOT SO FUNNY WHEN IT HAPPENS TO YOU? Once they...,Jacob Chabot,"John Cerilli, Frank Tieri",Nic Klein,Marvel Universe,Comic,Parental Advisory,$3.99,34987,34987 NOT SO FUNNY WHEN IT HAPPENS TO YOU? Onc...
23473,Zombie (2006),(2006),Zombie (2006) #4,"December 20, 2006",With a thousand zombies in front of him and tw...,Kyle Hotz,Mike Raicht,Kyle Hotz,MAX,Comic,EXPLICIT CONTENT,$3.99,34988,34988 With a thousand zombies in front of him ...
23474,Zombie (2006),(2006),Zombie (2006) #3,"November 29, 2006",The hordes of zombies gathered outside the hig...,Kyle Hotz,Mike Raicht,Kyle Hotz,MAX,Comic,EXPLICIT CONTENT,$3.99,34989,34989 The hordes of zombies gathered outside t...
23475,Zombie (2006),(2006),Zombie (2006) #2,"October 25, 2006","For Simon Garth, it's come down to two very ba...",Kyle Hotz,Mike Raicht,Kyle Hotz,MAX,Comic,EXPLICIT CONTENT,$3.99,34990,"34990 For Simon Garth, it's come down to two v..."


In [44]:
marvel_comics["tagged_description"] = (
    marvel_comics["tagged_description"]
    .str.replace(r"\s*\n\s*", " ", regex=True)
)

marvel_comics["tagged_description"].to_csv("tagged_description.txt",
                                           sep="\n",
                                           index=False)

In [45]:
raw_documents = TextLoader("tagged_description.txt", encoding='utf-8').load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 18, which is longer than the specified 0
Created a chunk of size 134, which is longer than the specified 0
Created a chunk of size 221, which is longer than the specified 0
Created a chunk of size 309, which is longer than the specified 0
Created a chunk of size 269, which is longer than the specified 0
Created a chunk of size 341, which is longer than the specified 0
Created a chunk of size 286, which is longer than the specified 0
Created a chunk of size 282, which is longer than the specified 0
Created a chunk of size 207, which is longer than the specified 0
Created a chunk of size 221, which is longer than the specified 0
Created a chunk of size 385, which is longer than the specified 0
Created a chunk of size 681, which is longer than the specified 0
Created a chunk of size 438, which is longer than the specified 0
Created a chunk of size 432, which is longer than the specified 0
Created a chunk of size 246, which is longer than the specified 0
Created a c

In [46]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='tagged_description')

In [47]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db_marvel_comics = Chroma.from_documents(documents, embedding=embeddings)

In [37]:
query = "A comic about the end of the world"
docs = db_marvel_comics.similarity_search(query, k=10)
docs

[Document(id='befcd966-750d-4c98-b8a7-59cd20f3a5c3', metadata={'source': 'tagged_description.txt'}, page_content="9434 Think the world is a bad place, everything is going to hell, and the end times are surely around the corner? Try reading the World's Greatest Comic Magazine."),
 Document(id='e6022125-852d-43fb-a228-18e62adc9bf8', metadata={'source': 'tagged_description.txt'}, page_content="9434 Think the world is a bad place, everything is going to hell, and the end times are surely around the corner? Try reading the World's Greatest Comic Magazine."),
 Document(id='f0ad8e71-8c68-49a8-869d-b09f192b6ea7', metadata={'source': 'tagged_description.txt'}, page_content="11373 The world has gone horribly wrong. A waking nightmare where a totalitarian despot rules a devastated countryside. Where mutated animals walk like men... and enslaved humanity crawls to avoid its master's whip. This is a world where justice fell into the sea with the California coastline, and hope is blanketed in the pe

In [48]:
marvel_comics[marvel_comics["id"] == int(docs[0].page_content.split()[0].strip())]

,comic_name,active_years,issue_title,publish_date,issue_description,penciler,writer,cover_artist,Imprint,Format,Rating,Price,id,tagged_description
6727,Fantastic Four (1998 - 2012),(1998 - 2012),Fantastic Four (1998) #579,"May 26, 2010","Think the world is a bad place, everything is ...",Neil Edwards,Jonathan Hickman,NaN,Marvel Universe,Comic,T,$2.99,9434,"9434 Think the world is a bad place, everythin..."


In [49]:
def retrieve_semantic_recommendation(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_marvel_comics.similarity_search(query, k=50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return marvel_comics[marvel_comics["id"].isin(books_list)].head(top_k)

In [50]:
retrieve_semantic_recommendation("A comic about the end of the world")

,comic_name,active_years,issue_title,publish_date,issue_description,penciler,writer,cover_artist,Imprint,Format,Rating,Price,id,tagged_description
0,A Year of Marvels: April Infinite Comic (2016),(2016),A Year of Marvels: April Infinite Comic (2016) #1,"April 01, 2016",The Infinite Comic that will have everyone tal...,Yves Bigerel,Yves Bigerel,Jamal Campbell,Marvel Universe,Infinite Comic,Rated T+,Free,0,0 The Infinite Comic that will have everyone t...
142,Age of Ultron (2013),(2013),Age of Ultron (2013) #10,"June 19, 2013",THE FINALE! The biggest secret in comics will ...,Brandon Peterson,"Brian Michael Bendis, Brandon Peterson",NaN,Marvel Universe,Comic,Rated T+,$3.99,200,200 THE FINALE! The biggest secret in comics w...
641,Amazing Spider-Man (1999 - 2013),(1999 - 2013),Amazing Spider-Man (1999) #682,"March 21, 2012","Part One of ""ENDS OF THE EARTH""! The Doc Ock s...",Stefano Caselli,Dan Slott,Stefano Caselli,Marvel Universe,Comic,T+,$3.99,938,"938 Part One of ""ENDS OF THE EARTH""! The Doc O..."
1979,Avengers Vs. X-Men (2012),(2012),Avengers Vs. X-Men (2012) #6.5,"June 25, 2012",Read the future of digital comics with the lat...,Carlo Barberi,Mark Waid,NaN,NaN,Infinite Comic,NaN,Free,2710,2710 Read the future of digital comics with th...
1986,Avengers Vs. X-Men (2012),(2012),Avengers Vs. X-Men (2012) #1.5,"April 04, 2012",Includes the groundbreaking AVENGERS VS. X-MEN...,NaN,NaN,NaN,NaN,Comic,NaN,$3.99,2717,2717 Includes the groundbreaking AVENGERS VS. ...
6727,Fantastic Four (1998 - 2012),(1998 - 2012),Fantastic Four (1998) #579,"May 26, 2010","Think the world is a bad place, everything is ...",Neil Edwards,Jonathan Hickman,NaN,Marvel Universe,Comic,T,$2.99,9434,"9434 Think the world is a bad place, everythin..."
7042,Fear Itself: Youth in Revolt (2011),(2011),Fear Itself: Youth in Revolt (2011) #6,"October 19, 2011",- The final fate of the new Initiative is reve...,NaN,Guisseppe Cammuncoli,NaN,NaN,Comic,NaN,$2.99,9848,9848 - The final fate of the new Initiative is...
8094,Heroes Reborn: Rebel (2000),(2000),Heroes Reborn: Rebel (2000) #1,"January 01, 2000",The world has gone horribly wrong. A waking ni...,"Scott Koblish, Matt Haley, Mark Bagley, Mark B...",Joe Kelly,Matt Haley,Marvel Universe,Comic,NaN,Free,11373,11373 The world has gone horribly wrong. A wak...
8120,History of the Marvel Universe (2019),(2019),History of the Marvel Universe (2019) #6,"December 18, 2019",From the stories of today to the end of the Ma...,Javier Rodriguez,"Mark Waid, Mike O'sullivan",Steve Mcniven,NaN,Comic,Rated T,$4.99,11401,11401 From the stories of today to the end of ...
8874,Infinity (2013),(2013),Infinity (2013) #1,"August 14, 2013",The oversized kickoff to the year's most antic...,Adam Kubert,Jonathan Hickman,NaN,Marvel Universe,Comic,Rated T+,$4.99,12456,12456 The oversized kickoff to the year's most...


In [51]:
retrieve_semantic_recommendation("A comic with a female main character")

,comic_name,active_years,issue_title,publish_date,issue_description,penciler,writer,cover_artist,Imprint,Format,Rating,Price,id,tagged_description
572,Amazing Fantasy (2004 - 2006),(2004 - 2006),Amazing Fantasy (2004) #3,"August 25, 2004",Enough with the questions -- now it's time for...,NaN,NaN,NaN,MARVEL UNIVERSE,Comic,MARVEL PSR,$2.99,862,862 Enough with the questions -- now it's time...
3999,Criminal 2 (2008),(2008),Criminal 2 (2008) #3,"June 04, 2008",It's the final of three interlinking standalon...,NaN,Sean Phillips,NaN,ICON,Comic,EXPLICIT CONTENT,$3.50,5691,5691 It's the final of three interlinking stan...
4867,Daughters of the Dragon: Deadly Hands Special ...,(2005),Daughters of the Dragon: Deadly Hands Special ...,"December 21, 2005",Re-presenting three classic tales of Marvel's ...,NaN,Chris Claremont,NaN,MARVEL UNIVERSE,Comic,T,$3.99,6891,6891 Re-presenting three classic tales of Marv...
7634,Girl Comics (2010),(2010),Girl Comics (2010) #1,"March 03, 2010",With the most controversial question in the Ma...,"Ming Doyle, Agnieszka Garbowska, Colleen Coove...","Trina Robbins, Colleen Coover, Devin Grayson, ...",NaN,NaN,Comic,NaN,$4.99,10791,10791 With the most controversial question in ...
11032,Marvel Rising (2018),(2018),Marvel Rising (2018) #1,"August 28, 2018",When Doreen Green (also known as the Unbeatabl...,Marco Failla,Devin Grayson,Helen Chen,NaN,Infinite Comic,All Ages,Free,15760,15760 When Doreen Green (also known as the Unb...
14389,Scarlet (2010 - 2016),(2010 - 2016),Scarlet (2010) #1,"July 08, 2010","THE AWARD WINNING, BEST SELLING, POWERHOUSE CR...",NaN,NaN,NaN,NaN,Comic,NaN,$3.95,21508,"21508 THE AWARD WINNING, BEST SELLING, POWERHO..."
15229,Spider-Girl (1998 - 2006),(1998 - 2006),Spider-Girl (1998) #80,"November 03, 2004",PART 1 (OF 1)\nA single-issue story and a grea...,Pat Olliffe,Tom Defalco,Ron Frenz,NaN,Comic,NaN,$2.99,22786,22786 PART 1 (OF 1) A single-issue story and a...
15871,Spider-Woman (2009 - 2010),(2009 - 2010),Spider-Woman (2009) #4,"December 23, 2009",It's Spider-Woman versus the Thunderbolts. Tha...,Alex Maleev,Brian Michael Bendis,NaN,MARVEL UNIVERSE,Comic,RATED T,$2.99,23725,23725 It's Spider-Woman versus the Thunderbolt...
15873,Spider-Woman (2009 - 2010),(2009 - 2010),Spider-Woman (2009) #2,"October 21, 2009",The Eisner award winning team of Bendis and Ma...,Alex Maleev,Brian Michael Bendis,NaN,MARVEL UNIVERSE,Comic,RATED T,$2.99,23727,23727 The Eisner award winning team of Bendis ...
18551,The Unbeatable Squirrel Girl (2015 - 2019),(2015 - 2019),The Unbeatable Squirrel Girl (2015) #41,"February 13, 2019","In the past months, Squirrel Girl has battled ...",Naomi Franquiz,Ryan North,Erica Henderson,NaN,Comic,Rated T,$3.99,27360,"27360 In the past months, Squirrel Girl has ba..."


In [52]:
retrieve_semantic_recommendation("A comic about revenge")

,comic_name,active_years,issue_title,publish_date,issue_description,penciler,writer,cover_artist,Imprint,Format,Rating,Price,id,tagged_description
0,A Year of Marvels: April Infinite Comic (2016),(2016),A Year of Marvels: April Infinite Comic (2016) #1,"April 01, 2016",The Infinite Comic that will have everyone tal...,Yves Bigerel,Yves Bigerel,Jamal Campbell,Marvel Universe,Infinite Comic,Rated T+,Free,0,0 The Infinite Comic that will have everyone t...
12,A Year of Marvels: The Uncanny (2016),(2016),A Year of Marvels: The Uncanny (2016) #1,"December 07, 2016",It's Halloween in the Marvel U! What does that...,NaN,NaN,NaN,Marvel Universe,Comic,Rated T+,$4.99,12,12 It's Halloween in the Marvel U! What does t...
5507,Destroyer (2009),(2009),Destroyer (2009) #2,"May 06, 2009","""DESTROYER,"" PART 2 The clock is ticking, Dest...",Cory Walker,Robert Kirkman,NaN,MAX,Comic,EXPLICIT CONTENT,$3.99,7809,"7809 ""DESTROYER,"" PART 2 The clock is ticking,..."
8073,Heroes for Hire (2010 - 2011),(2010 - 2011),Heroes for Hire (2010) #5,"April 06, 2011","Guest-starring The Falcon, Iron Fist and The P...",Robert Atkins,"Dan Abnett, Andy Lanning",Doug Braithwaite,NaN,Comic,NaN,$2.99,11351,"11351 Guest-starring The Falcon, Iron Fist and..."
8094,Heroes Reborn: Rebel (2000),(2000),Heroes Reborn: Rebel (2000) #1,"January 01, 2000",The world has gone horribly wrong. A waking ni...,"Scott Koblish, Matt Haley, Mark Bagley, Mark B...",Joe Kelly,Matt Haley,Marvel Universe,Comic,NaN,Free,11373,11373 The world has gone horribly wrong. A wak...
8431,Identity Disc (2004),(2004),Identity Disc (2004) #4,"November 08, 2004",Blackmailed into the service of a mysterious w...,NaN,NaN,NaN,Marvel Universe,Comic,Marvel Psr,$2.99,11828,11828 Blackmailed into the service of a myster...
9913,Kick-Ass (2008 - 2009),(2008 - 2009),Kick-Ass (2008) #2,"April 02, 2008",Sickening violence...just the way you like it....,John Romita,Mark Millar,NaN,ICON,Comic,EXPLICIT CONTENT,$2.99,13955,13955 Sickening violence...just the way you li...
12960,Not Brand Echh (1967 - 1969),(1967 - 1969),Not Brand Echh (1967) #2,"September 01, 1967",Who says a comic book has to be good?? Featuri...,"Don Heck, Marie Severin","Roy Thomas, Stan Lee, Gary Friedrich",NaN,Marvel Universe,Comic,NaN,Free,19115,19115 Who says a comic book has to be good?? F...
14750,Sgt. Fury (1963 - 1974),(1963 - 1974),Sgt. Fury (1963) #19,"June 01, 1965","While on furlough, Sgt. Fury decides to seek r...",NaN,Jack Kirby,NaN,NaN,Comic,NaN,Free,22087,"22087 While on furlough, Sgt. Fury decides to ..."
17336,Superior Spider-Man (2013 - 2014),(2013 - 2014),Superior Spider-Man (2013) #9,"May 01, 2013","IT ENDS HERE! """"SUPERIOR NO MORE!"""" The hottes...",Marcos Martin,Dan Slott,NaN,Marvel Universe,Comic,Rated T,$3.99,25596,"25596 IT ENDS HERE! """"SUPERIOR NO MORE!"""" The ..."
